### Jane Street Puzzle: Beside the Point

I solved this puzzle from Jane Street earned a spot on their leaderboard all the way back in November, but I procrastinated posting my solution in detail until now. This notebook explains my approach to solving the puzzle. I ran a simulation first to have a good idea of the solution, but since the solution required at least 10 decimal places, an analytical approach was needed to place on the leaderboard. Below, I've explained the problem first, explained my simulation code, and then finally my mathematical solution.

- **Here's the official puzzle:** [Beside the Point](https://www.janestreet.com/puzzles/beside-the-point-index/)
- **And here's my name on the leaderboard:** [Solution & Leaderboard](https://www.janestreet.com/puzzles/beside-the-point-solution/)



In [1]:
import numpy as np
from scipy.integrate import dblquad

#### Problem Overview

Consider the unit square $[0,1]^2$. We can pick two points uniformly at random:
- **A blue point:** $(x_1, y_1)$
- **And a red point:** $(x_2, y_2)$

We want the probability that there exists a point *on the boundary of the square* that is equidistant 
to these two chosen points. By symmetry, I'll focus on just one particular edge (say, the bottom edge) 
and one particular subset of blue points. Specifically, assume the blue point lies in the portion of 
$[0,1]^2$ for which the bottom side is the closest edge. This subset has area $\tfrac{1}{4}$ of the entire square.




#### Equidistance Condition

For the bottom edge to harbor a point equidistant to both $(x_1, y_1)$ and $(x_2, y_2)$, consider a candidate point $(a, 0)$ on the bottom side. The equidistance requirement is:

$$
(x_1 - a)^2 + y_1^2 \;=\; (x_2 - a)^2 + y_2^2.
$$

After expanding and simplifying, I can solve for $a$:

$$
a \;=\; \frac{x_2^2 - x_1^2 + y_2^2 - y_1^2}{2 \,\bigl(x_2 - x_1\bigr)}.
$$

For the pair $(x_1, y_1)$ and $(x_2, y_2)$ to be *successful*, the value of $a$ must lie between 0 and 1, 
i.e., $0 \le a \le 1$.


### Vectorized Simulation Approach

To estimate this probability numerically:

1. I restrict blue points to the region where the bottom edge is indeed the closest edge. 
   By symmetry, this region occupies $\tfrac{1}{4}$ of the unit square. I'm generating these points via rejection sampling.
2. I pick red points uniformly from the entire unit square $[0,1]^2$.
3. For each pair, I compute the candidate $a$ and check if $0 \le a \le 1$.
4. The fraction of successful pairs (out of all valid pairs) directly gives the conditional probability, 
   which, due to symmetry, coincides with the overall probability we're looking for.

Below is a vectorized implementation of these steps in NumPy.


In [2]:
def vectorized_simulation(num_samples: int) -> float:
    # generate blue points (bottom-closest region) via rejection sampling.
    blue = []
    while len(blue) < num_samples:
        pts = np.random.rand(num_samples, 2)
        valid = (
            (pts[:, 1] <= 0.5) &    # y ≤ 0.5
            (pts[:, 1] <= pts[:, 0]) &    # y ≤ x
            (pts[:, 1] <= 1 - pts[:, 0])  # y ≤ 1 - x
        )
        blue.extend(pts[valid].tolist())
    blue = np.array(blue[:num_samples])
    
    # generate red points uniformly in [0,1]^2.
    red = np.random.rand(num_samples, 2)
    
    # exclude pairs where x2 ≈ x1 to avoid division by zero in 'a'.
    mask = ~np.isclose(red[:, 0], blue[:, 0], atol=1e-8)
    
    a_vals = (
        (red[mask, 0]**2 - blue[mask, 0]**2 + red[mask, 1]**2 - blue[mask, 1]**2)
        / (2 * (red[mask, 0] - blue[mask, 0]))
    )
    
    successes = np.sum((a_vals >= 0) & (a_vals <= 1))
    probability = successes / np.sum(mask)
    
    return probability


#### Simulation Results

Now, I'll run the simulation with a large sample size to approximate this probability. 
Since the simulation draws a pretty large number of points, it should provide a 
decently accurate numerical estimate.


In [3]:
num_samples = 10**6  # one million samples
estimated_probability = vectorized_simulation(num_samples)
print(f"Estimated Probability from Simulation: {estimated_probability:.10f}")

Estimated Probability from Simulation: 0.4916058825


### Analytical Solution Explanation

In addition to the Monte Carlo simulation, we can get an exact (or exact-enough) solution by examining the geometry of the problem in detail.

#### Setup and Geometry

- WLOG, restrict the first (blue) point to the region of the unit square where the bottom edge is the nearest boundary. This region is one-quarter of the square.
- For a fixed blue point $(x_1, y_1)$, I'm looking for all $(x_2, y_2)$ in $[0,1]^2$ for which there exists a point on the bottom edge $(a,0)$ equidistant to both.
- Geometrically, when $(x_1, y_1)$ is fixed, the set of valid $(x_2, y_2)$ is the symmetric difference of two circles:
   - *Circle 1*: center $(0,0)$, radius $r_1 = \sqrt{x_1^2 + y_1^2}$.
   - *Circle 2*: center $(1,0)$, radius $r_2 = \sqrt{(1 - x_1)^2 + y_1^2}$.

#### Analytical Calculation of the Region

- Each circle, if considered alone within the unit square, cuts out a quarter-circle area relevant to valid positions for $(x_2, y_2)$. However, where the two circles overlap, I have to subtract the intersection to avoid double-counting.

- The overlap of the two circular arcs is determined by chord geometry and the angles subtended by the radii. These angles can be computed using inverse cosine relationships. The area of intersection then comprises specific circular sectors minus the triangular-like segments formed by the chord.

- After determining the valid red-point area for each fixed blue point, I'll integrate over all possible $(x_1, y_1)$ within the restricted one-quarter region. Symbolically,
$$
\text{Probability} 
\;=\;
4 \,\times
\int_{\text{blue region}} 
\frac{\text{Area of valid red points}}{1}
\; d(x_1, y_1),
$$
where the factor of 4 is copmensating for having confined the blue point to one-quarter of the unit square.

In [4]:
# distance functions
def r1(x, y):
    return np.sqrt(x**2 + y**2)

def r2(x, y):
    return np.sqrt((1 - x)**2 + y**2)

# helper functions for computing angles for the overlapping quarter-circles
def c_ad(x, y):
    rr1 = r1(x, y)
    rr2 = r2(x, y)
    if rr1 == 0:
        return 0
    arg = (rr1**2 + 1 - rr2**2) / (2 * rr1)
    arg = np.clip(arg, -1, 1)
    return 2 * np.arccos(arg)

def c_bd(x, y):
    rr1 = r1(x, y)
    rr2 = r2(x, y)
    if rr2 == 0:
        return 0
    arg = (rr2**2 + 1 - rr1**2) / (2 * rr2)
    arg = np.clip(arg, -1, 1)
    return 2 * np.arccos(arg)

# intersection area of the two quatrer-circles
def intersection_area(x, y):
    rr1 = r1(x, y)
    rr2 = r2(x, y)
    if rr1 == 0 or rr2 == 0:
        return 0
    cad = c_ad(x, y)
    cbd = c_bd(x, y)
    return 0.5 * (rr1**2 * (cad - np.sin(cad)) + rr2**2 * (cbd - np.sin(cbd)))

# valid red point area for a given blue point x y
def A(x, y):
    return (np.pi * r1(x, y)**2)/4 + (np.pi * r2(x, y)**2)/4 - intersection_area(x, y)


In [5]:
# integration
I1, err1 = dblquad(
    lambda yy, xx: A(xx, yy),
    0, 0.5,      # xx in [0, 0.5]
    lambda xx: 0,
    lambda xx: xx
)

I2, err2 = dblquad(
    lambda yy, xx: A(xx, yy),
    0.5, 1,      # xx in [0.5, 1]
    lambda xx: 0,
    lambda xx: 1 - xx
)

P = 4 * (I1 + I2)
print("Actual Probability:", P)


Actual Probability: 0.491407578838308



#### Final Probability

From this, we get that:
$$
\text{Probability} 
\;=\; 
\frac{1 + 2\pi - \ln(4)}{12}
\;\approx\; 
0.4914075788.
$$

Therefore, both the analytical integration and the numerical simulation agree on the final probability of roughly $0.4914$.


Contact me: akalekar@purdue.edu